In [11]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option("display.max_columns", None)
from sklearn.cluster import KMeans

In [2]:
client_id = "f68226fa347346169cb5cdec05f72a8b"
client_secret ="e082fbec5984468f946e9e056e9083ce"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
song_data = pd.read_csv("/Users/aloaskari/Desktop/Ironhack/Bootcamp/Week_6/Song_recommender/Data/songs.csv")

In [4]:
song_data.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist,hot_100
0,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,audio_features,0yLdNVWF3Srea0uzk55zFn,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://api.spotify.com/v1/audio-analysis/0yLd...,200455,4,flowers,miley cyrus,1
1,0.347,0.378,8,-9.960,1,0.0354,0.8570,0.000000,0.1180,0.610,173.482,audio_features,5ReIs3aH4wfBjp93QOtWAl,spotify:track:5ReIs3aH4wfBjp93QOtWAl,https://api.spotify.com/v1/tracks/5ReIs3aH4wfB...,https://api.spotify.com/v1/audio-analysis/5ReI...,168609,4,kill bill,sza,1
2,0.517,0.675,6,-5.382,1,0.0357,0.4590,0.000000,0.1510,0.518,203.853,audio_features,59uQI0PADDKeE6UZDTJEe8,spotify:track:59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://api.spotify.com/v1/audio-analysis/59uQ...,163855,4,last night,morgan wallen,1
3,0.714,0.472,2,-7.375,1,0.0864,0.0130,0.000005,0.2660,0.238,131.121,audio_features,3nqQXoyQOWXiESFLlDF1hG,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...,156943,4,unholy,sam smith & kim petras,1
4,0.728,0.293,1,-13.657,0,0.0562,0.4790,0.904000,0.1180,0.152,98.302,audio_features,4CpujwKsq4DxOIMiKr12m8,spotify:track:4CpujwKsq4DxOIMiKr12m8,https://api.spotify.com/v1/tracks/4CpujwKsq4Dx...,https://api.spotify.com/v1/audio-analysis/4Cpu...,217026,4,creepin',"metro boomin, the weeknd & 21 savage",1


In [8]:
song_data.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'title', 'artist', 'hot_100'],
      dtype='object')

In [13]:
def recommend_song(input_song, song_database_file, n_clusters=50):
    results = sp.search(input_song, limit=1, type='track')
    if len(results['tracks']['items']) == 0:
        print('Could not find audio features for input song.')
        return None
    input_id = results['tracks']['items'][0]['id']
    input_features = sp.audio_features(input_id)[0]

    features = song_data.drop(['title', 'artist'], axis=1).values
    kmeans = KMeans(n_clusters=n_clusters).fit(features)

    cluster_id = kmeans.predict([input_features.drop(['id', 'uri', 'track_href', 'analysis_url', 'type', 'duration_ms'])])[0]
    cluster_songs = song_data[kmeans.labels_ == cluster_id]
    features = cluster_songs.drop(['title', 'artist'], axis=1).values
    input_vector = input_features.drop(['id', 'uri', 'track_href', 'analysis_url', 'type', 'duration_ms', 'audio_features']).values.reshape(1, -1)
    sims = cosine_similarity(features, input_vector)

    cluster_songs['similarity'] = sims
    cluster_songs = cluster_songs.sort_values(by='similarity', ascending=False)

    if len(cluster_songs) > 0:
        recommendation = cluster_songs.sample(n=1)['song_name'].values[0]
        return recommendation
    else:
        print('Could not find a similar song in the same cluster.')
        return None

In [14]:
input_song = "Mask Off"
song_database_file = "/Users/aloaskari/Desktop/Ironhack/Bootcamp/Week_6/Song_recommender/Data/songs.csv"
recommended_song = recommend_song(input_song, song_database_file)
print(f"Recommended song: {recommended_song}")


ValueError: could not convert string to float: 'audio_features'